In [ ]:
# default_exp core

# autograd_simpler

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

### Functions 

In [ ]:
#export 
def somefun(x): 
    """x is a number, fool"""
    return x**2

In [ ]:
somefun(4)

16

In [ ]:
assert somefun(5) == 25

### Classes 

In [ ]:
#export
class Box: 
    """A box for values"""
    def __init__(self, x): self.x = x
    def say(self):
        "Apply `somefun` to `x` and print it out" 
        return somefun(self.x)
    

In [ ]:
show_doc(Box.say)

<h4 id="Box.say" class="doc_header"><code>Box.say</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>Box.say</code>()

Apply [`somefun`](/autograd_simpler/core#somefun) to `x` and print it out

In [ ]:
b1 = Box(4)
b1.say()

16

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
